In [1]:
import nbsetup
import ergodicpy as ep
import numpy as np
import seaborn as sns
import scipy as sp
import pandas as pd
from simulations.snowflake.models import Snowflake
import matplotlib.pyplot as plt

In [ ]:
ensembles = 50
spikes = 3
trials = 5

def gen_flakes(count):
    ms = []
    for i in range(count):
        nbsetup.cp(count-i)
        ms.append(Snowflake(plot=False, spikes=spikes))
    return ms

models = gen_flakes(ensembles)
trials = gen_flakes(trials)

19


In [ ]:
states = 360
histograms = [s.histogram(states=states) for s in models]
ec = ep.ErgodicCollection(histograms)
ec.complexity

In [ ]:
data = []

for t, s in enumerate(trials):
    final_pmf = s.histogram(states=states)
    ergodic_pmf = ec.ergodic_pmf()
    
    for d in range(10, s.steps, 50):
        nbsetup.cp(t, d)
        current_pmf = s.histogram(states=states, at_point=d)
        
        compare = sp.spatial.distance.jensenshannon
        dd = {
            'step': d,
            'trial': t,
            'actual difference (current x final)': compare(current_pmf, final_pmf),
            'frequentist (ergodic x final)': compare(ergodic_pmf, final_pmf),
        }
        
        for i in range(5):
            swarm_pmf = ec.swarm_pmf(current_pmf, silent=True, mode=i)
            dd['swarm (%s)' % i] = compare(swarm_pmf, final_pmf)
        
        data.append(dd)



In [ ]:
df = pd.DataFrame(data=data)
sns.lineplot(data=df.melt(id_vars=['step', 'trial']), x='step', y='value', hue='variable')